# CUDA exercises

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [1]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df

In [2]:
def calcGB_Stride(df):
    return 3*8* df["N"]/df["k"] /1e9

def calcGB_Offset(df):
    return 3*8* (df["N"]-df["k"])/1e9

def transform_to_ms(df, cols):
    return df[cols]*1e3


# Ex7.1

In [8]:
df_CPU = csv_parse("data/ph_data_CPU.csv", header=False)
display(df_CPU)
df_GPU = csv_parse("data/ph_data_GPU.csv", header=False)
display(df_GPU)
df_cuda = csv_parse("data/data_cuda.csv", header=False)
display(df_cuda)
data = [df_CPU, df_GPU, df_cuda]

,N,target,local_size,global_size,ocl_time,cpu_time,total_time,dot
0,256,CPU,128,16384,0.000068,0.753478,0.753546,512.0
1,1000,CPU,128,16384,0.000055,0.756965,0.757020,2000.0
2,10000,CPU,128,16384,0.000077,0.802797,0.802874,20000.0
3,100000,CPU,128,16384,0.000081,0.772425,0.772506,200000.0
4,1000000,CPU,128,16384,0.000844,0.762968,0.763812,2000000.0
5,10000000,CPU,128,16384,0.017677,0.756647,0.774324,20000000.0
6,100000000,CPU,128,16384,0.216136,0.773296,0.989432,200000000.0


,N,target,local_size,global_size,ocl_time,cpu_time,total_time,dot
0,256,GPU,128,16384,0.000026,0.728097,0.728123,512.0
1,1000,GPU,128,16384,0.000028,0.742840,0.742868,2000.0
2,10000,GPU,128,16384,0.000028,0.738157,0.738185,20000.0
3,100000,GPU,128,16384,0.000025,0.736451,0.736476,200000.0
4,1000000,GPU,128,16384,0.000086,0.728222,0.728308,2000000.0
5,10000000,GPU,128,16384,0.000685,0.729255,0.729940,20000000.0
6,100000000,GPU,128,16384,0.006479,0.744036,0.750515,200000000.0


,N,blocks,block_size,tests_done,total_time,time_per_test,check
0,256,256,256,10,0.408000,0.040800,0
1,1000,256,256,10,0.432549,0.043255,0
2,10000,256,256,10,0.396001,0.039600,0
3,100000,256,256,10,0.428198,0.042820,0
4,1000000,256,256,10,0.444003,0.044400,0
5,10000000,256,256,10,0.426076,0.042608,0
6,100000000,256,256,10,0.847996,0.084800,0


In [9]:
Ns = df_CPU["N"].unique()
Ns

array([      256,      1000,     10000,    100000,   1000000,  10000000,
       100000000])

In [10]:
def print_styling():

    plt.rc('figure', figsize=(14,8)) 
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('lines', linewidth=2)

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
print_styling()

In [ ]:
fig, axs = plt.subplots(figsize=(14,12))

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

print_styling();
for n in N:
    for df in data:
        df.plot(x="N", y="total_time", ax=ax)

axs[0][0].set_ylabel("runtime [s]")
axs[1][0].set_ylabel("runtime [s]")
axs[1][0].set_xlabel("N")
axs[1][1].set_xlabel("N")
#df.plot(x=x, y=gpu_ref+["time_cpuref"], logx=logx, logy=logy, linestyle="--", ax=ax);



fig.suptitle("Sparse times Dense Matrix: Runtimes")
plt.tight_layout()

#fig.savefig("spTd_runtimes.pdf", bbox_inches="tight", pad_inches=0.2)